<hr>

# 🤖 05 - MACHINE LEARNING 🤖

<style>
h1 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

### 📂 IMPORTs


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [2]:
# install xgboost if not already installed
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# install shap if not already installed
!pip install shap

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# install numpy and pandas if not already installed
!pip install numpy pandas

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)



<hr>

# 0️⃣ Load + Preprocess Dataset + Test

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

## 📊 STEP 1 — Load Data


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [5]:
import pandas as pd
# load the clean dataset
df = pd.read_csv("../data/processed/online_shoppers_intention_07_encoded.csv")

# For consistency with code:
    # X = features without the target variable
    # y = target variable "revenue"

# drop the target variable "revenue" from the features
X = df.drop("revenue", axis=1)

# target variable "revenue" is already binary (0 and 1) 
y=df["revenue"]

# if it's not already in that format (it should be, but just to be safe)
# y = df["revenue"].astype(int)


<hr>

## 🧹 STEP 2 — Preprocessing


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [6]:
# identify numerical and categorical features for preprocessing

    # - NUMERICAL features will be scaled for models that require it (Logistic Regression, SVM, KNN), 
    # but not for tree-based models (Decision Tree, Random Forest, XGBoost)
numeric_features = X.select_dtypes(include=['int64','float64']).columns

    # - CATEGORICAL features will be one-hot encoded for all models
    # we include bool type as well since we have some boolean features that should be treated as categorical
categorical_features = X.select_dtypes(include=['object','bool']).columns

print(f"Identified {len(numeric_features)} numerical features and {len(categorical_features)} categorical features for preprocessing.")
print(f"Numerical features: {list(numeric_features)}")
print(f"Categorical features: {list(categorical_features)}")


# define two separate preprocessors:  ------------------------------------------------    
    # - one with scaling for models that require it
preprocessor_scaled = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])
    # - one without scaling for tree-based models
preprocessor_tree = ColumnTransformer([
    ('num', 'passthrough', numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])
print("Defined two separate preprocessors: one with scaling for models that require it, and one without scaling for tree-based models.")

Identified 74 numerical features and 0 categorical features for preprocessing.
Numerical features: ['admin', 'admin_duration', 'info', 'info_duration', 'prod_related', 'prod_related_duration', 'bounce_rate', 'exit_rate', 'page_value', 'special_day', 'weekend', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jul', 'month_June', 'month_Mar', 'month_May', 'month_Nov', 'month_Oct', 'month_Sep', 'visitor_type_New_Visitor', 'visitor_type_Other', 'visitor_type_Returning_Visitor', 'browser_1', 'browser_2', 'browser_3', 'browser_4', 'browser_5', 'browser_6', 'browser_7', 'browser_8', 'browser_9', 'browser_10', 'browser_11', 'browser_12', 'browser_13', 'os_1', 'os_2', 'os_3', 'os_4', 'os_5', 'os_6', 'os_7', 'os_8', 'region_1', 'region_2', 'region_3', 'region_4', 'region_5', 'region_6', 'region_7', 'region_8', 'region_9', 'traffic_type_1', 'traffic_type_2', 'traffic_type_3', 'traffic_type_4', 'traffic_type_5', 'traffic_type_6', 'traffic_type_7', 'traffic_type_8', 'traffic_type_9', 'traffic_type_10'

```text
the num vs cat is wrong don't use
it's just not useable
```

<hr>

## ✂️ STEP 3 — Train/Test Split


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

| Variable         | Description                                      | Role in Modeling                 |
|------------------|--------------------------------------------------|-----------------------------------|
| `X`                | session features                                 | Independent variables (input)     |
| `y`                | Revenue outcome (purchase intention)             | Dependent variable (target)       |
| `features_train`  | Training feature set used to train models             | Model training data                |
| `features_test`   | Testing feature set used to evaluate model performance | Model evaluation data             |
| `target_train`    | Training target labels  | Supervised learning - Binary target (train)|
| `target_test`     | Testing target labels      | Supervised learning - Binary target (test) |

In [7]:
# Split the data into:
# - Training set
# - Test set
# Using stratify to maintain class balance

(
    features_train,     # features for training
    features_test,      # features for testing
    target_train,       # target for training
    target_test         # target for testing
) = train_test_split(
    X, # features without the target variable "revenue"
    y, # target variable "revenue"
    test_size=0.2,      # 20% test set, 80% train set
    stratify=y,         # maintain class balance in splits
    random_state=42     # for reproducibility, we can choose any integer as the random state
)



# save the train and test sets to CSV files for later use in modeling
features_train.to_csv("../data/processed/features_train.csv", index=False)
print("[features_train] saved to CSV file.")

features_test.to_csv("../data/processed/features_test.csv", index=False)
print("[features_test] saved to CSV file.")



# save the target variable for train and test sets to CSV files for later use in modeling
target_train.to_csv("../data/processed/target_train.csv", index=False)
print("[target_train] saved to CSV file.")

target_test.to_csv("../data/processed/target_test.csv", index=False)
print("[target_test] saved to CSV file.")

[features_train] saved to CSV file.
[features_test] saved to CSV file.
[target_train] saved to CSV file.
[target_test] saved to CSV file.


<hr>

# 1️⃣ BASELINE

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

In [8]:
# For consistency in code team collaboration 
# assign:  X_train,  X_test,  y_train,  y_test 
X_train = features_train
X_test = features_test
y_train = target_train
y_test = target_test

print(70*"-")

# calculate the class imbalance ratio for the training set to use in XGBoost's scale_pos_weight parameter
ratio = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Class imbalance ratio (negative:positive) in training set: {ratio:.2f}")
    # This ratio will be used in the XGBoost model to help it handle the class imbalance effectively.
    # A ratio greater than 1 indicates that the negative class (0) is more prevalent than the positive class (1),  which is common in many real-world datasets. 
    # By setting scale_pos_weight to this ratio, we can help XGBoost give more attention to the minority class during training.

print(70*"-")

# distribution ENTIRE dataset
print("Class distribution in entire dataset:")
print(pd.concat([y_train, y_test]).value_counts())
# percentage ENTIRE dataset
print("\nClass distribution percentages in entire dataset:")
print(pd.concat([y_train, y_test]).value_counts(normalize=True) * 100)
print(70*"-")

# distribution TRAIN set
print("Class distribution in training set:")
print(y_train.value_counts())
# percentage TRAIN set
print("\nClass distribution percentages in training set:")
print(y_train.value_counts(normalize=True) * 100)
print(70*"-")


# distribution TEST set
print("Class distribution in test set:")
print(y_test.value_counts())
# percentage TEST set
print("/nClass distribution percentages in test set:")
print(y_test.value_counts(normalize=True) * 100)
print(70*"-")




# Class imbalance ratio (for XGBoost) ---------------------------------------------------------------------------
pos_count = (y_train == 1).sum()
neg_count = (y_train == 0).sum()

if pos_count == 0:
    ratio = 1
else:
    ratio = neg_count / pos_count

print(f"Class imbalance ratio (negative:positive) in training set: {ratio:.2f}")
print(70 * "-")

----------------------------------------------------------------------
Class imbalance ratio (negative:positive) in training set: 5.40
----------------------------------------------------------------------
Class distribution in entire dataset:
revenue
0    10297
1     1908
Name: count, dtype: int64

Class distribution percentages in entire dataset:
revenue
0    84.367063
1    15.632937
Name: proportion, dtype: float64
----------------------------------------------------------------------
Class distribution in training set:
revenue
0    8238
1    1526
Name: count, dtype: int64

Class distribution percentages in training set:
revenue
0    84.371159
1    15.628841
Name: proportion, dtype: float64
----------------------------------------------------------------------
Class distribution in test set:
revenue
0    2059
1     382
Name: count, dtype: int64
/nClass distribution percentages in test set:
revenue
0    84.350676
1    15.649324
Name: proportion, dtype: float64
-----------------------

In [9]:
# Cross-Validation F1 Mean & Std  -------------------------------------------------------------------------------------------

# F1 Score Mean and Standard Deviation across folds for the last model
print("F1 Score Mean and Standard Deviation across folds for the last model:")
print(f"Mean: {np.mean(f1_list):.4f}") # mean of F1 scores across the 5 folds
print(f"Standard Deviation: {np.std(f1_list):.4f}") # standard deviation of F1 scores across the 5 folds

F1 Score Mean and Standard Deviation across folds for the last model:


NameError: name 'f1_list' is not defined

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    matthews_corrcoef, confusion_matrix
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier


# ==========================================
# Class imbalance ratio (for XGBoost)
# ==========================================
pos_count = (y_train == 1).sum()
neg_count = (y_train == 0).sum()
ratio = neg_count / pos_count if pos_count != 0 else 1

print(f"Imbalance ratio (neg:pos) = {ratio:.2f}")
print(70 * "-")


# ==========================================
# Define Models
# ==========================================
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Linear SVM": SVC(kernel="linear", probability=True),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(
        eval_metric='logloss',
        scale_pos_weight=ratio,
        random_state=42
    ),
    "AdaBoost": AdaBoostClassifier(random_state=42)
}


# ==========================================
# Cross-validation setup
# ==========================================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []


# ==========================================
# Train + Evaluate
# ==========================================
for name, model in models.items():

    # ---- Fit on training data
    model.fit(X_train, y_train)

    # ---- Predictions on test set
    y_pred = model.predict(X_test)

    # ---- Probabilities (for AUC metrics)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_proba = model.decision_function(X_test)

    # ---- Metrics (Test Set)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    pr_auc = average_precision_score(y_test, y_proba)
    mcc = matthews_corrcoef(y_test, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # ---- Cross-validation (F1)
    cv_scores = cross_val_score(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring="f1"
    )

    mean_cv = cv_scores.mean()
    std_cv = cv_scores.std()

    # ---- Store Results
    results.append([
        name, precision, recall, f1,
        (y_test == 1).sum(),   # Support (positive class)
        pr_auc, roc_auc,
        mean_cv, std_cv,
        mcc,
        tp, tn, fp, fn
    ])


# ==========================================
# Create Results Table
# ==========================================
columns = [
    "Model", "Precision", "Recall", "F1 Score",
    "Support", "PR-AUC", "ROC-AUC",
    "Mean (CV F1)", "Std (CV F1)",
    "MCC",
    "TP", "TN", "FP", "FN"
]

results_df = pd.DataFrame(results, columns=columns)

print(results_df.sort_values(by="F1 Score", ascending=False))
print(70 * "-")

<hr>

## 📐 STEP 4 — Define Models


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [10]:
# define a dictionary of models with appropriate preprocessing and class imbalance handling ------------------------------------------
models = {

    # Logistic Regression can handle class imbalance by setting class_weight='balanced',
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Logistic Regression": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", LogisticRegression(class_weight='balanced', max_iter=1000))
    ]),
    
    # Linear SVM can handle class imbalance by setting class_weight='balanced', 
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Linear SVM": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", SVC(kernel='linear', class_weight='balanced', probability=True))
    ]),
    
    # KNN doesn't have a built-in way to handle class imbalance, but it can still be effective.
    # We will use the preprocessor with scaling for KNN as well, since KNN is sensitive to feature scales.
    "KNN": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", KNeighborsClassifier())
    ]),
    
    # Decision Tree can handle class imbalance by setting class_weight='balanced',
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Decision Tree": Pipeline([
        ("prep", preprocessor_tree),
        ("model", DecisionTreeClassifier(class_weight='balanced'))
    ]),
    
    # Random Forest can handle class imbalance by setting class_weight='balanced', 
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Random Forest": Pipeline([
        ("prep", preprocessor_tree),
        ("model", RandomForestClassifier(class_weight='balanced'))
    ]),
    
    # XGBoost has a built-in way to handle class imbalance using the scale_pos_weight parameter.
    "XGBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", XGBClassifier(scale_pos_weight=ratio, eval_metric='logloss'))
    ]),

    # AdaBoost doesn't have a built-in way to handle class imbalance, but it can still be effective. 
    # We will use the preprocessor without scaling for AdaBoost as well.
    "AdaBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", AdaBoostClassifier())
    ])
}

print("Defined the following models with appropriate preprocessing and class imbalance handling:")
for model_name in models.keys():
    print(f" - {model_name}")

Defined the following models with appropriate preprocessing and class imbalance handling:
 - Logistic Regression
 - Linear SVM
 - KNN
 - Decision Tree
 - Random Forest
 - XGBoost
 - AdaBoost


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

models = {
    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=5000))
    ]),
    
    "Linear SVM": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="linear", probability=True))
    ]),
    
    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier())
    ]),

    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    "AdaBoost": AdaBoostClassifier()
}

<hr>

## ⚡ STEP 5 —  Run Baseline CV Comparison


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    matthews_corrcoef, confusion_matrix
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier


# Define Models------------------------------------------------------------------------------------------------

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Linear SVM": SVC(kernel="linear", probability=True),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    "AdaBoost": AdaBoostClassifier()
}

# Cross Validation Setup------------------------------------------------------------------------------------------------

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

# Loop Through Models------------------------------------------------------------------------------------------------

for name, model in models.items():

    precision_list = []
    recall_list = []
    f1_list = []
    roc_list = []
    pr_list = []
    mcc_list = []

    tp_total = tn_total = fp_total = fn_total = 0

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Probabilities for AUC
        y_proba = model.predict_proba(X_test)[:, 1]

        # Metrics
        precision_list.append(precision_score(y_test, y_pred))
        recall_list.append(recall_score(y_test, y_pred))
        f1_list.append(f1_score(y_test, y_pred))
        roc_list.append(roc_auc_score(y_test, y_proba))
        pr_list.append(average_precision_score(y_test, y_proba))
        mcc_list.append(matthews_corrcoef(y_test, y_pred))

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        tp_total += tp
        tn_total += tn
        fp_total += fp
        fn_total += fn

    results.append([
        name,
        np.mean(precision_list),# Precision 
        np.mean(recall_list),   # Recall
        np.mean(f1_list),       # F1 Score
        np.sum(y == 1),         # Support (positive class)
        np.mean(pr_list),       # PR-AUC
        np.mean(roc_list),      # ROC-AUC
        np.mean(f1_list),       # Performance (using F1)
        np.std(f1_list),        # Standard deviation
        np.mean(mcc_list),      # MCC
        tp_total, tn_total, fp_total, fn_total # Confusion Matrix totals
    ])


# Create Results Table ------------------------------------------------------------------------------------------------

columns = [
    "Model", "Precision", "Recall", "F1 Score",
    "Support", "PR-AUC", "ROC-AUC",
    "Mean (F1)", "Std (F1)",
    "MCC", "TP", "TN", "FP", "FN"
]

results_df = pd.DataFrame(results, columns=columns)

print(results_df)

C:\Users\sboub\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\sboub\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable

: 

In [ ]:
# cross-validation to evaluate the models using ROC AUC as the scoring metric, which is appropriate for imbalanced classification problems
cv_results = {
    "KNN": cross_val_score(models["KNN"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc'),
    "Logistic Regression": cross_val_score(models["Logistic Regression"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc'),
    "Linear SVM": cross_val_score(models["Linear SVM"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc'),
    "Decision Tree": cross_val_score(models["Decision Tree"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc'),
    "Random Forest": cross_val_score(models["Random Forest"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc'),
    "XGBoost": cross_val_score(models["XGBoost"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc'),
    "AdaBoost": cross_val_score(models["AdaBoost"], X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='roc_auc')
}


In [ ]:
display(cv_results)

In [ ]:
# use stratified k-fold cross-validation to evaluate baseline performance of each model
# cv = cross validation strategy that maintains class balance in each fold, which is important for imbalanced datasets like ours.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# store mean ROC-AUC scores for each model in results dictionary
results = {
    # after cross-validation:
    # the model names as keys and their mean ROC-AUC scores as values 
    "Logistic Regression": None,
    "Linear SVM": None,
    "KNN": None,
    "Decision Tree": None,
    "Random Forest": None,
    "XGBoost": None,
    "AdaBoost": None
}

# loop through each model, perform cross-validation, and store the mean ROC-AUC score
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train,
                             cv=cv, scoring='roc_auc')
    results[name] = scores.mean()

# create a DataFrame to display the results in a sorted manner
baseline_results = pd.DataFrame.from_dict(results, orient='index',
                                          columns=['ROC-AUC']).sort_values(by="ROC-AUC", ascending=False)

# display the baseline results
print("Baseline ROC-AUC scores for each model:")
display(baseline_results)

---
#### NOTES
👉 Best model by ROC-AUC: XGBoost (0.9275)
- It has the highest ability to rank positive cases above negative cases.

⚠️ Is ROC-AUC enough for imbalanced data? Short answer: No, it is not enough.
- ROC-AUC can be misleading for imbalanced datasets because:
    - It evaluates performance across all classification thresholds.
    - It gives equal importance to True Positive Rate (TPR) and False Positive Rate (FPR).
    - When negatives heavily outnumber positives, the FPR can look artificially small, inflating ROC-AUC.
    - In highly imbalanced data, a model can achieve high ROC-AUC but still perform poorly at identifying the minority class.

🔎 For imbalanced datasets, prioritize:

1️⃣ Precision-Recall AUC (PR-AUC) ⭐ (Very Important)
- More informative when the positive class is rare.
- Focuses only on the positive class performance.

2️⃣ F1-score
- Harmonic mean of precision and recall.
- Good when you need balance between false positives and false negatives.

3️⃣ Recall (Sensitivity)
- Important if missing positives is costly (e.g., fraud, disease).

4️⃣ Confusion Matrix
- Shows actual classification behavior at your chosen threshold.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict
from sklearn.metrics import confusion_matrix

cross_validation_metrics = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scoring = {
    'roc_auc': 'roc_auc',
    'pr_auc': 'average_precision',  # PR-AUC
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

results = {}

for name, model in models.items():
    
    # Cross-validated metrics
    cv_scores = cross_validate(
        model,
        X_train,
        y_train,
        cv=cross_validation_metrics,
        scoring=scoring,
        return_train_score=False
    )
    
    # Predictions for confusion matrix
    y_pred = cross_val_predict(
        model,
        X_train,
        y_train,
        cv=cross_validation_metrics
    )
    
    results[name] = {
        "ROC-AUC": cv_scores['test_roc_auc'].mean(),
        "PR-AUC": cv_scores['test_pr_auc'].mean(),
        "Accuracy": cv_scores['test_accuracy'].mean(),
        "Precision": cv_scores['test_precision'].mean(),
        "Recall": cv_scores['test_recall'].mean(),
        "F1-score": cv_scores['test_f1'].mean(),
        "Confusion Matrix": confusion_matrix(y_train, y_pred)
    }

# Print results
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    for metric, value in metrics.items():
        print(f"{metric}:\n{value}")

<hr>

## 🏆 STEP 6 —  Select TOP 3 Models


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### MODELS - METRICS


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
import pandas as pd

# import metrics for evaluation on the test set
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


# define a function to evaluate a model on the test set and return a dictionary of metrics
def evaluate_model(model, model_name):
    # Fit model
    model.fit(features_train, target_train)
    
    # Predictions
    y_pred = model.predict(features_test)
    y_prob = model.predict_proba(features_test)[:, 1]
    
    # Metrics (Buy = positive class = 1)
    accuracy = accuracy_score(target_test, y_pred)
    precision_buy = precision_score(target_test, y_pred)
    recall_buy = recall_score(target_test, y_pred)
    f1_buy = f1_score(target_test, y_pred)
    roc_auc = roc_auc_score(target_test, y_prob)
    
    return {
        "Model": model_name,
        "Accuracy": round(accuracy, 3),
        "Precision (Buy)": round(precision_buy, 3),
        "Recall (Buy)": round(recall_buy, 3),
        "F1-score (Buy)": round(f1_buy, 3),
        "ROC-AUC": round(roc_auc, 3)
    }

In [ ]:
results = []

results.append(evaluate_model(knn_model, "KNN"))
results.append(evaluate_model(logreg_model, "Logistic Regression"))

comparison_df = pd.DataFrame(results)

print("\nModel Comparison")
print(comparison_df.sort_values(by="ROC-AUC", ascending=False))

<hr>

### CONFUSION MATRIX


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
cm = confusion_matrix(target_test, y_pred)

cm_table = pd.DataFrame(
    cm,
    index=["Actual Not Buy", "Actual Buy"],
    columns=["Pred Not Buy", "Pred Buy"]
)

print(f"\nConfusion Matrix - {model_name}")
print(cm_table)

In [ ]:
cm_2 = confusion_matrix(y_test, y_train)

cm_table_2 = pd.DataFrame(
    cm_2,
    index=["Actual Not Buy", "Actual Buy"],
    columns=["Pred Not Buy", "Pred Buy"]
)

print(f"\nConfusion Matrix - {model_name}")
print(cm_table_2)

<hr>

# 2️⃣ DEEP TUNING TOP 3 MODELS

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

## MODEL 1:


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

## MODEL 2:


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

## MODEL 3:


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

# 3️⃣ FINAL EVALUATION

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

In [ ]:
# replace with best_model from "DEEP TUNING TOP 3" phase
# for example, if Random Forest was the best model after tuning, you would set:
best_model = best_rf

# fit the best model on the entire training set
best_model.fit(X_train, y_train)

# evaluate on test set
# get predicted probabilities for ROC-AUC and predicted classes for other metrics

# y_pred will be used for F1 score, confusion matrix, and classification report
y_pred = best_model.predict(X_test)
# y_prob will be used for ROC-AUC
y_prob = best_model.predict_proba(X_test)[:,1]

# print evaluation metrics
print("Test ROC-AUC:", roc_auc_score(y_test, y_prob))
# print F1 score
print("F1 Score:", f1_score(y_test, y_pred))
# print confusion matrix 
print(confusion_matrix(y_test, y_pred))
# print classification report
print(classification_report(y_test, y_pred))

<hr>

# 🏁 Final Model Selection Criteria

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

Choose model with:
- Highest CV ROC-AUC
- Good F1
- Good Recall (important for purchase detection)
- Stable performance (low variance across folds)

<hr>

# 📊 PLOTS

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

## 01 - ROC Curves (Compare top 3 models)


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
# paste the plot version you want to add to main notebook here

<hr>

### 01 - ROC Curves - Model 1


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 01 - ROC Curves - Model 2


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 01 - ROC Curves - Model 3


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

## 02 - Adjust Decision Threshold


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- **Instead of 0.5:**
- **This should improve recall.**

In [ ]:
# paste the plot version you want to add to main notebook here

<hr>

### 02 -Adjust Decision Threshold - V1


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 02 - Adjust Decision Threshold - V2


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

## 03 - Feature Importance


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- **For RF / XGBoost: Use  `.feature_importances_`**

In [ ]:
import matplotlib.pyplot as plt